***GENERATED CODE FOR predicthighestincomeregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Occupation'}], 'feature': 'Occupation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '55', 'mean': '', 'stddev': '', 'min': 'ALL OCCUPATIONS', 'max': 'Wholesale and retail buyers, except farm products', 'missing': '0', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Occupation'}, {'transformation_label': 'String Indexer', 'feature_label': 'Occupation'})
        transformationDF = transformationDF.drop('Occupation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'M_weekly'}], 'feature': 'M_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '55', 'mean': '1369.61', 'stddev': '314.92', 'min': '1006', 'max': 'Na', 'missing': '0', 'distinct': '37'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'M_weekly'}, {'transformation_label': 'String Indexer', 'feature_label': 'M_weekly'})
        transformationDF = transformationDF.drop('M_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'F_weekly'}], 'feature': 'F_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '55', 'mean': '1066.85', 'stddev': '236.7', 'min': '1002', 'max': 'Na', 'missing': '0', 'distinct': '34'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'F_weekly'}, {'transformation_label': 'String Indexer', 'feature_label': 'F_weekly'})
        transformationDF = transformationDF.drop('F_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'All_weekly'}], 'feature': 'All_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '55', 'mean': '1213.86', 'stddev': '287.16', 'min': '1002', 'max': 'Na', 'missing': '0', 'distinct': '42'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'All_weekly'}, {'transformation_label': 'String Indexer', 'feature_label': 'All_weekly'})
        transformationDF = transformationDF.drop('All_weekly')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


def randomforestregressor(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestRegressor()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))
    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run predicthighestincomeregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	predicthighetincometraindata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1666682052117_Predict Highest Income Train.csv', 'FilePath': '/Finance Models/Predict Highest Income- Regression/Predict Highest Income Train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'Predict Highest Income Train.csv', 'viewFileName': 'Predict Highest Income Train.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://dev.numtra.com:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run predicthighestincomeregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(predicthighetincometraindata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Occupation"}], "feature": "Occupation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "55", "mean": "", "stddev": "", "min": "ALL OCCUPATIONS", "max": "Wholesale and retail buyers, except farm products", "missing": "0", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Occupation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "M_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "55", "mean": "1463.93", "stddev": "8209.34", "min": "5", "max": "60746", "missing": "0"}, "updatedLabel": "M_workers"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "M_weekly"}], "feature": "M_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "55", "mean": "1369.61", "stddev": "314.92", "min": "1006", "max": "Na", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "M_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "F_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "55", "mean": "1177.89", "stddev": "6527.16", "min": "2", "max": "48334", "missing": "0"}, "updatedLabel": "F_workers"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "F_weekly"}], "feature": "F_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "55", "mean": "1066.85", "stddev": "236.7", "min": "1002", "max": "Na", "missing": "0", "distinct": "34"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "F_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "All_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "55", "mean": "2641.95", "stddev": "14734.41", "min": "8", "max": "109080", "missing": "0"}, "updatedLabel": "All_workers"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "All_weekly"}], "feature": "All_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "55", "mean": "1213.86", "stddev": "287.16", "min": "1002", "max": "Na", "missing": "0", "distinct": "42"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "All_weekly"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run predicthighestincomeregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=randomforestregressor(manualfe, ["Occupation_stringindexer", "M_workers", "M_weekly_stringindexer", "F_workers", "F_weekly_stringindexer", "All_workers"], "All_weekly_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
